Try to GET data from the web. If no response, retry n_tries times with exponential backoff.

In [1]:
possibly_get_data <- function(url, n_tries, ...){

    rate <- rate_backoff(pause_base = 1, max_times = n_tries)
    possibly_insistent_get <- insistently(GET, rate, quiet = FALSE) %>% possibly(otherwise = NULL)

    possibly_insistent_get(url, ...)
}

Check to see if a station_elevation is present, and if NULL, use the geonames.org webservice to provide an elevation for a given latitude and longitude

In [ ]:
findStationElevation <- function(station_elevation, station_latitude, station_longitude, geonames_username) {
    if (is.null(station_elevation)) { 
        station_elevation <- possibly_get_data(
            paste0("http://api.geonames.org/gtopo30JSON?lat=", station_latitude, 
                   "&lng=", station_longitude, "&username=", 
                   geonames_username), 8) %>% content %>% pluck("gtopo30")}
    station_elevation
}

In [ ]:
displayNearbyCity <- function(station_latitude, station_longitude, geonames_username) {
    possibly_get_data(paste0("http://api.geonames.org/findNearbyPlaceNameJSON?lat=", 
                                              station_latitude, "&lng=", station_longitude, "&username=", 
                                              geonames_username), 8) %>% content
}

Find the offset for the internal clock on the dongle. Going to use the `kal` command and for this we need to set the gain, so first check to see if that's in the config.  Then use `kal` to find a nearby GSM station in the GSM900 band with a default offset of 0. Then use that station to test the offset

In [98]:
findClockOffset <- function(clock_offset, gain) {
    if (is.null(gain)) {
        gain <- findDongleMaxGain()
    }
    if (is.null(clock_offset)) {
        print("No clock offset found. Running kal. This will take a few minutes")
        flush.console()
        best_channel <- system(paste0("kal -s GSM900 -e 0 -g ", gain, " | sed 's/ \\+/\\t/g' | cut -f 3,8 | ",
                              "sort -k2 -n -r | head -1 | cut -f 1"),
                       intern = TRUE)
        print("GSM station found, testing dongle drift")
        flush.console()
        clock_offset <- system(paste0("kal -c ", best_channel, " -g ", gain, " -e 0"), intern=TRUE) %>%
            pluck(5) %>% strsplit(":") %>% pluck(1,2) %>% substr(2,nchar(.) - 4) %>% as.numeric
    }
    return(clock_offset)
}

Parse the output of rtl_test to find the maximum gain of the dongle

In [92]:
findDongleMaxGain <- function(gain) {
    if (is.null(gain)) {
        rtl_test_output <- system2('timeout', c('2', 'rtl_test'), stdout = TRUE, stderr = TRUE)
        if(any(grepl("Failed to open", rtl_test_output))) {
            print("Unable to open RTL SDR device. Is it busy or missing?")
            stop()
        }
        if(any(grepl("Supported gain", rtl_test_output))) {
            rtl_test_output %>% pluck(grep("Supported", .)) %>% strsplit(" ") %>% 
                unlist %>% tail(n=1) %>% as.numeric
        }
    }
    return(gain)
}